# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,selebi-phikwe,-21.9790,27.8498,295.44,51,76,4.35,BW,1699120966
1,1,yarmouth,41.7057,-70.2286,290.35,58,0,4.63,US,1699120966
2,2,fort bragg,35.1390,-79.0060,296.29,39,0,2.57,US,1699120966
3,3,uturoa,-16.7333,-151.4333,298.54,84,88,8.09,PF,1699120967
4,4,howard springs,-12.4970,131.0470,300.10,86,99,2.39,AU,1699120967


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
# Extract latitude, longitude, and humidity data from city_data_df
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )



# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Convert temperature from Kelvin to Celsius
city_data_df['Temp_Celsius'] = city_data_df['Max Temp'] - 273.15

# Convert temperature from Kelvin to Celsius and store it in a new column
city_data_df['Temp_Celsius'] = city_data_df['Max Temp'] - 273.15

In [20]:
# Narrow down cities that fit criteria and drop any results with null values

# Criteria:
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

weather_filtered_df = city_data_df.loc[(city_data_df["Temp_Celsius"] < 27)
                                       & (city_data_df["Temp_Celsius"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Temp_Celsius
2,2,fort bragg,35.1390,-79.0060,296.29,39,0,2.57,US,1699120966,23.14
88,88,guichon,-32.3500,-57.2000,294.77,41,0,3.52,UY,1699120992,21.62
95,95,laguna,38.4210,-121.4238,297.39,48,0,3.60,US,1699120994,24.24
116,116,andrews,32.3187,-102.5457,298.38,27,0,2.57,US,1699120988,25.23
118,118,west modesto,37.6175,-121.0391,296.67,53,0,2.06,US,1699121000,23.52
164,164,lompoc,34.6391,-120.4579,296.40,36,0,2.57,US,1699120805,23.25
202,202,nova sintra,14.8667,-24.7167,297.06,73,1,4.24,CV,1699121020,23.91
212,212,kailua-kona,19.6406,-155.9956,299.04,81,0,0.45,US,1699120951,25.89
214,214,ramhormoz,31.2800,49.6035,298.47,39,2,2.21,IR,1699121022,25.32
221,221,reguiba,33.5639,6.7033,296.18,38,0,1.03,DZ,1699121024,23.03


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,fort bragg,US,35.1390,-79.0060,39,
88,guichon,UY,-32.3500,-57.2000,41,
95,laguna,US,38.4210,-121.4238,48,
116,andrews,US,32.3187,-102.5457,27,
118,west modesto,US,37.6175,-121.0391,53,
164,lompoc,US,34.6391,-120.4579,36,
202,nova sintra,CV,14.8667,-24.7167,73,
212,kailua-kona,US,19.6406,-155.9956,81,
214,ramhormoz,IR,31.2800,49.6035,39,
221,reguiba,DZ,33.5639,6.7033,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
        }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: No hotel found
guichon - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
andrews - nearest hotel: No hotel found
west modesto - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
nova sintra - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
ramhormoz - nearest hotel: No hotel found
reguiba - nearest hotel: No hotel found
rizokarpaso - nearest hotel: No hotel found
al faw - nearest hotel: No hotel found
bechar - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
cocal - nearest hotel: No hotel found
ghadamis - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
2,fort bragg,US,35.1390,-79.0060,39,No hotel found
88,guichon,UY,-32.3500,-57.2000,41,No hotel found
95,laguna,US,38.4210,-121.4238,48,No hotel found
116,andrews,US,32.3187,-102.5457,27,No hotel found
118,west modesto,US,37.6175,-121.0391,53,No hotel found
164,lompoc,US,34.6391,-120.4579,36,No hotel found
202,nova sintra,CV,14.8667,-24.7167,73,No hotel found
212,kailua-kona,US,19.6406,-155.9956,81,No hotel found
214,ramhormoz,IR,31.2800,49.6035,39,No hotel found
221,reguiba,DZ,33.5639,6.7033,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols=["City", "Country", "Hotel Name"]
                                  ).opts(tools=["hover"], toolbar="above")

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)